# Lab 9 Part 3 /
# Connecting to a Microsoft Azure SQL Database with pyodbc

In [22]:
from sqlalchemy import create_engine
import textwrap
import pyodbc
import getpass
import pandas as pd
import numpy as np

#### Setting up the connection string (database location) and the query variables

In [23]:
sql_query_1 = """
SELECT last_name, first_name,
    count(rental_id) AS number_of_films_rented_may
FROM rental a
    INNER JOIN customer b ON a.customer_id = b.customer_id
WHERE MONTH(rental_date) = 5
GROUP BY last_name, first_name
ORDER BY 1;
"""

In [24]:
sql_query_2 = """
SELECT *
	INTO rentals_may
FROM rental
WHERE month(rental_date) = 5;
"""

In [25]:
sql_query_3 = """
SELECT * INTO rentals_june
FROM rental
WHERE month(rental_date) = 6;
"""

In [26]:
sql_query_4 = """
select top(5) *
from rentals_june;
"""

In [27]:
# Driver: find the DRIVERS that are available using the pyodbc.drivers() method
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [28]:
# Driver:
driver='{ODBC Driver 17 for SQL Server}'

# Server name and DataBase name:
server_name='tcp:serv-sakila-2'
database_name='sakila'

# Creating the Server URL:
server='{server_name}.database.windows.net,1433'.format(server_name=server_name)

In [29]:
# User name and password
username='a100jcd'
token=getpass.getpass()

········


In [30]:
# Setting up the Azure SQL database connection

DATABASE_LOCATION = textwrap.dedent('''
    Driver={driver};
    Server={server};
    Database={database};
    Uid={username};
    Pwd={token};
    Encrypt=yes;
    TrustServerCertificate=no;
    Connection Timeout=30;
    '''.format(driver=driver,server=server, 
    database=database_name, username=username, token=token))

#### Connect and extract data

In [31]:
cnx = pyodbc.connect(DATABASE_LOCATION)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'serv-sakila-2' requested by the login. Client with IP address '2.206.133.211' is not allowed to access the server.  To enable access, use the Windows Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. (40615) (SQLDriverConnect); [42000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'serv-sakila-2' requested by the login. Client with IP address '2.206.133.211' is not allowed to access the server.  To enable access, use the Windows Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this chang

In [11]:
data_1 = pd.read_sql_query(sql_query_1, cnx)
data_1.head()

,last_name,first_name,number_of_films_rented_may
0,ABNEY,RAFAEL,2
1,ADAM,NATHANIEL,3
2,ADAMS,KATHLEEN,2
3,ALLARD,GORDON,1
4,ALLEN,SHIRLEY,1


In [192]:
cursor = cnx.cursor()

In [193]:
cursor.execute(sql_query_2)

In [194]:
cursor.execute(sql_query_3)

In [201]:
data_2 = pd.read_sql_query(sql_query_4, cnx)

In [202]:
data_2.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1185,2005-06-15 00:54:00,2785,1,2005-06-23 02:42:00,2,2006-02-15 21:30:00
1,1354,2005-06-15 13:13:00,1294,83,2005-06-23 13:08:00,2,2006-02-15 21:30:00
2,1523,2005-06-16 00:18:00,681,349,2005-06-17 02:50:00,2,2006-02-15 21:30:00
3,1692,2005-06-16 12:30:00,3544,514,2005-06-17 17:31:00,2,2006-02-15 21:30:00
4,1861,2005-06-17 01:17:00,135,454,2005-06-25 02:11:00,1,2006-02-15 21:30:00
